In [1]:
import pymc3 as pm

In [2]:
with pm.Model() as model:

    retrieve_utility = pm.Uniform('U_r', lower=0, upper=1)
    payment = retrieve_utility * pm.Uniform('k_p', lower=0, upper=1)

    prior_retrieve_probability_with_deal = pm.Uniform('P_{r,d}', lower=0.99, upper=1.0)s
    prior_retrieve_probability_without_deal = prior_retrieve_probability_with_deal * pm.Uniform('k_d', lower=0.5, upper=1)

    prior_slash_probability = 1 - prior_retrieve_probability_with_deal
    prior_expected_appeal_count = prior_slash_probability

    deposit_multiplier = pm.Uniform('m_d', 0, 1000)
    slash_multiplier= pm.Uniform('m_s', 0, deposit_multiplier)
    appeal_multiplier = pm.Uniform('m_a', 0, slash_multiplier)
    deal_refused_cost = pm.Uniform('c_d', lower=0, upper=0.1)

    propose_payoff = retrieve_utility * prior_retrieve_probability_with_deal
    propose_payoff -= payment * (1 - prior_slash_probability)
    propose_payoff -= appeal_multiplier * payment * prior_expected_appeal_count
    propose_payoff -= deal_refused_cost

    no_proposal_payoff = retrieve_utility * prior_retrieve_probability_without_deal
    
    # Best Response Criteria
    propose_probability = propose_payoff > no_proposal_payoff
    D_1 = pm.Bernoulli('D_1', p=propose_probability)

    slash_probability = pm.Uniform('P(S)', lower=0, upper=0.001)
    accept_payoff = payment * (1 - slash_probability)
    accept_payoff -= slash_multiplier * payment * slash_probability

    refuse_payoff = 0.0

    accept_probability = accept_payoff > refuse_payoff
    D_2 = pm.Bernoulli('D_2', p=accept_probability)

    posterior = pm.sample(tune=1000, draws=500, return_inferencedata=True)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [P(S), c_d, m_a, m_s, m_d, k_d, P_{r,d}, k_p, U_r]
>BinaryGibbsMetropolis: [D_1, D_2]


Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 20 seconds.
/opt/homebrew/Caskroom/miniconda/base/lib/python3.9/site-packages/arviz/stats/diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
There were 183 divergences after tuning. Increase `target_accept` or reparameterize.
There were 185 divergences after tuning. Increase `target_accept` or reparameterize.
There were 157 divergences after tuning. Increase `target_accept` or reparameterize.
There were 177 divergences after tuning. Increase `target_accept` or reparameterize.


In [3]:
posterior

Inference data with groups:
	> posterior
	> sample_stats